# Introduction to functional data cleaning using nilearn

### Movement is the enemy of neuroimagers

In task-based fMRI, if participants move during task-relevant moments
- Get a huge **false task-related signal** that’s actually due to motion!

In resting-state fMRI, movement can induce **false correlations** between brain regions


Solving for this involves *modelling* our fMRI signal to be comprised of **true brain signal** and **confounder signals**. 


Our goal is to remove a majority (hopefully) of the **confounder signals** and acquire something *closer* to the **true signal**. 

This is achieved via **confound regression**, which is essentially fitting a linear model using confounds as regressors then subtracting it out from the signal


*****

In [ ]:
import os
from nilearn import image as img
from nilearn import plotting as plot
import matplotlib.pyplot as plt
%matplotlib inline

## Step 1: Implementing Confound Regression using FMRIPREP outputs
FMRIPREP estimates confounds for the functional image and outputs it into:

**sub-xxxxx_task-xxxx_space-xxxx_..._confounds.tsv**

Let's load one up and see what it looks like

In [ ]:
func_dir = '../data/func/'
os.listdir(func_dir)

The confounds file is organized like an excel spread-sheet with multiple columns, each for a specific confound.

We can view these using pandas 

In [ ]:
import pandas as pd

In [ ]:
confound = os.path.join(func_dir,
                        'sub-10206_task-rest_bold_confounds.tsv'
                        )
confound_df = pd.read_csv(confound,delimiter='\t')

In [ ]:
confound_df.head()

Each of these confounds is computed automatically by fmriprep. The choice of which confounds to use in functional imaging analysis is a source of large debate. We recommend that you check out these sources for a start:

1. https://www.sciencedirect.com/science/article/pii/S1053811917302288#f0005
2. https://www.sciencedirect.com/science/article/pii/S1053811917302288

For now we're going to replicate the pre-processing (mostly) from the seminal Yeo1000 17-networks paper:

https://www.ncbi.nlm.nih.gov/pubmed/21653723

### The Full Yeo 2011 Pre-processing schema

#### Confound regressors
1. 6 motion parameters (X, Y, Z, RotX, RotY, RotZ) 
2. Global signal 
3. White matter signal
4. CSF Signal

This is a total of 9 base confound regressor variables. Finally adds temporal derivatives of each of these signals as well (1 temporal derivative for each), the result is 18 confound regressors.

#### Low/High pass filtering
1. Low pass filtering cutoff: 0.08 
2. High pass filtering cutoff: 0.009

Low pass filters out high frequency signals from our data. fMRI signals are slow evolving processes, any high frequency signals are likely due to noise 
High pass filters out any very low frequency signals (below 0.009Hz), which may be due to intrinsic scanner instabilities

#### Drop dummy TRs
During the initial stages of a functional scan there is a strong signal decay artifact, thus the first 4ish or so 
TRs are very high intensity signals that don't reflect the rest of the scan. Therefore we drop these TRs. 